# Test whether: programmers who created entries and spent more than 3 hours in the same entry are those who have more entries without notes.

## Read df with users "any notes"

In [174]:
import pandas as pd

In [175]:
#read df with only users with one or more entries "without any notes"
dfusers1 = pd.read_csv(r"C:\Users\marin\Downloads\jupyter\users1_before_groupby")
dfusers1

,month,user_id,brokenruledescription,hours,user_type
0,1,4001436.0,Time entry without any notes.,1.285000,1
1,1,4001436.0,The time entry is a manual entry running,0.280000,1
2,1,3997286.0,Time entry without any notes.,2.146154,1
3,1,3997286.0,The time entry is a manual entry stopped,0.640000,1
4,1,3997286.0,Time entry bigger than 3 hours.,4.030000,1
...,...,...,...,...,...
1030,12,1852591.0,Time entry without any notes.,8.628000,1
1031,12,1852591.0,Time entry bigger than 3 hours.,6.757692,1
1032,12,1852573.0,Time entry without any notes.,5.159091,1
1033,12,1852573.0,The time entry is a manual entry stopped,0.420000,1


In [176]:
#groupby user
dfusers1 = dfusers1.groupby(['user_id']).agg({'hours':'sum'}).reset_index()

In [177]:
#get only users ids
dfusers1 = dfusers1[['user_id']].copy()

## Read database and clean it up

In [178]:
#read database
sheet_id = '1VK2ZQyVCNNjh4D-aQ4xyRjU8DrsLhQ9O02FnZ9S_nSU'
df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")

In [179]:
#clean database
#nan values
df = df.dropna(how="any", axis=0) 
#values equal to 0
df = df[(df != 0).all(1)]

In [180]:
#create new df with important columns 
dftask3 = df[['user_id', 'brokenruledescription', 'hours', 'spent_date']].copy()

In [181]:
dftask3['month'] = pd.to_datetime(df['spent_date']).dt.month
dftask3 = dftask3.drop(['spent_date'], axis = 1)

## Classify users with one or more entries "Time entry bigger than 3 hours"

In [182]:
#filter the users with at least one "Time entry bigger than 3 hours."
dftask3 = dftask3.groupby(['month','user_id','brokenruledescription']).agg({'hours':'mean'}).reset_index()

In [183]:
#classify entry type. Type 1 is for those who entered one or more "Time entry bigger than 3 hours.", else, type -1.
dftask3['bigger'] = [1 if d == 'Time entry bigger than 3 hours.' else -1 for d in dftask3['brokenruledescription']] 

In [184]:
#classify users by entrys classification
dftask3 = dftask3.sort_values(["month", "user_id","bigger"], ascending=[True, False,False])

user_temp = ''

for i in dftask3.index: 
    if(dftask3["user_id"][i] == user_temp):
        tipoUser = dftask3["bigger"][i]
        dftask3["bigger"][i] = tipoUserTemp

    user_temp = dftask3["user_id"][i]
    tipoUserTemp = dftask3["bigger"][i]
    

<ipython-input-184-ab056596f4cf>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftask3["bigger"][i] = tipoUserTemp


In [185]:
dfbigger = dftask3.groupby(['user_id']).agg({'hours':'sum'}).reset_index().drop(['hours'], axis = 1)

## Compare dfusers1 to dfbigger 

In [186]:
#if user id in dfusers1 is also in dfbigger then column "exist" is filled with True, else, "False" 
dfusers1['exist'] = dfusers1['user_id'].isin(dfbigger['user_id'])

In [187]:
#True and False percentages  in dataframe
dfusers1['exist'].value_counts(normalize=True) * 100

True     90.384615
False     9.615385
Name: exist, dtype: float64